# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [35]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [36]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    ##Uncomment the code below to check the files retrived.
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [37]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print('There are {} rows in the overall list of rows'.format(len(full_data_rows_list)))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


There are 8056 rows in the overall list of rows


In [38]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print('There are {} rows in the csv file -- including headers'.format(sum(1 for line in f)))

There are 6821 rows in the csv file -- including headers


# Part II. Data Modeling and Inserting Data Into Tables 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [39]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [40]:
# TO-DO: Create a Keyspace 
try:
    session.execute(
        """CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [41]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

###Check keyspace creation
chk_query='''
SELECT * FROM system_schema.tables 
WHERE keyspace_name = 'sparkify';
'''
[print(x) for x in session.execute(chk_query)]

[]

## Create tables to support three different data access patterns

### These are examples of the data access patterns we will use.

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Table Creation process for the first data access pattern

In [42]:
'''
Check to see if the test data is in the csv file prior to inserting into table
'''
df=pd.read_csv('event_datafile_new.csv')
df[(df['sessionId']==338)&(df['itemInSession']==4)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
5250,Faithless,Ava,F,4,Robinson,495.3073,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50


#### Create first table music_sessionId which supports queries of the music app history by session Id

In [43]:
#drop table if exists -- facilitates a clean setup for multiple testing
session_table_drop = "DROP TABLE IF EXISTS music_sessionId"
try:
    session.execute(session_table_drop)
except Exception as e:
    print(e)
    
create_query='CREATE TABLE sparkify.music_sessionId '
create_query=create_query+ ''' (sessionId int, itemInSession int, artist text, song_title text, song_length decimal, PRIMARY KEY(sessionId, itemInSession))
WITH CLUSTERING ORDER BY (itemInSession ASC);''' 
try:
    session.execute(create_query)
except Exception as e:
        print(e)                    

#### Model the data in the csv file and insert into the music_sessionId table 

In [44]:
'''
Process the csv file to create the first table
'''

file = 'event_datafile_new.csv'
chk=0

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        chk+=1
        insert_query = "INSERT INTO music_sessionId ( itemInSession, sessionId, artist, song_title, song_length)"
        insert_query = insert_query + "VALUES (%s,%s,%s,%s,%s)"

        session.execute(insert_query,(int(line[3]),int(line[8]),line[0],line[9],float(line[5])))

print('{} rows in csv'.format(chk))

###Query table to confirm the same number of rows exist there
query='''
SELECT count(*) FROM music_sessionId;
'''

rows=session.execute(query)
for row in rows:
    print(*rows)

6820 rows in csv
Row(count=6820)


#### Do a SELECT to verify that the data have been inserted into each table

In [45]:
## Verify the data was entered into the table
query='''
SELECT artist,song_title,song_length FROM sparkify.music_sessionId WHERE sessionid=338 AND iteminsession=4;
'''


try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows: 
    print((row.artist, row.song_title, row.song_length))

('Faithless', 'Music Matters (Mark Knight Dub)', Decimal('495.3073'))


## Table Creation Process for the second data access pattern

In [46]:
'''
This table suports the data access pattern to find music listened to by a particular user during a specific session
'''
##CREATE TABLE
session_table_drop1 = "DROP TABLE IF EXISTS artist_song_user"
try:
    session.execute(session_table_drop1)
except Exception as e:
    print(e)
    
create_query1='CREATE TABLE IF NOT EXISTS artist_song_user '
create_query1=create_query1+ ''' (userid int, sessionId int, itemInSession int, artist text, song_title text, firstName text, lastName text, 
PRIMARY KEY((userid, sessionid),itemInSession))
WITH CLUSTERING ORDER BY (itemInSession ASC);''' 
try:
    session.execute(create_query1)
except Exception as e:
        print(e)                    

##INSERT the modeled data into the table
file = 'event_datafile_new.csv'
chk=0

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        chk+=1
        insert_query1 = "INSERT INTO artist_song_user (userId, sessionId, itemInSession, artist, song_title, firstName, lastName)"
        insert_query1 = insert_query1 + " VALUES (%s,%s,%s,%s,%s,%s,%s)"

        session.execute(insert_query1,
                        (int(line[10]), int(line[8]), int(line[3]),line[0], line[9],  line[1], line[4])
                       )

print('There are {} rows in csv'.format(chk))

###Query table to confirm the same number of rows exist there
query1='''SELECT count(*) FROM artist_song_user;'''

rows=session.execute(query1)
for row in rows:
    print(*rows)

    
## Verify the data was entered into the table
query='''SELECT artist,iteminsession,song_title,firstname,lastname FROM sparkify.artist_song_user WHERE userid=10 AND sessionid=182;'''


try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows: 
    print((row.artist, row.iteminsession ,row.song_title, row.firstname, row.lastname))                    

There are 6820 rows in csv
Row(count=6820)
('Down To The Bone', 0, "Keep On Keepin' On", 'Sylvie', 'Cruz')
('Three Drives', 1, 'Greece 2000', 'Sylvie', 'Cruz')
('Sebastien Tellier', 2, 'Kilometer', 'Sylvie', 'Cruz')
('Lonnie Gordon', 3, 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', 'Sylvie', 'Cruz')


## Table Creation Process for the third data access pattern

In [47]:
##CREATE TABLE
session_table_drop2 = "DROP TABLE IF EXISTS user_song"
try:
    session.execute(session_table_drop2)
except Exception as e:
    print(e)
    
create_query2='CREATE TABLE IF NOT EXISTS user_song '
create_query2=create_query2+ ''' (song_title text, userId int, firstName text, lastName text, 
PRIMARY KEY(song_title,userId))
WITH CLUSTERING ORDER BY (userId ASC);''' 
try:
    session.execute(create_query2)
except Exception as e:
        print(e)                    

##INSERT modeled data into the table
file = 'event_datafile_new.csv'
chk=0

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        chk+=1
        insert_query2 = "INSERT INTO user_song (song_title, userId, firstName, lastName)"
        insert_query2 = insert_query2 + " VALUES (%s,%s,%s,%s)"

        session.execute(insert_query2,
                        (line[9], int(line[10]),line[1], line[4])
                       )


print('There are {} rows in csv'.format(chk))

###Query table to confirm the same number of rows exist there
query1='''SELECT count(*) FROM user_song;'''

rows=session.execute(query1)
for row in rows:
    print(*rows)

    
## Verify the data was entered into the table
query='''SELECT song_title,firstname,lastname FROM sparkify.user_song WHERE song_title='All Hands Against His Own';'''


try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows: 
    print((row.song_title, row.firstname, row.lastname))       

                    

There are 6820 rows in csv
Row(count=6618)
('All Hands Against His Own', 'Jacqueline', 'Lynch')
('All Hands Against His Own', 'Tegan', 'Levine')
('All Hands Against His Own', 'Sara', 'Johnson')


### Drop the tables before closing out the sessions

In [48]:
## Drop the table before closing out the sessions
drop_query1='''
DROP TABLE music_sessionId
'''
drop_query2='''
DROP TABLE artist_song_user
'''
drop_query3='''
DROP TABLE user_song
'''
drop_tables=[drop_query1,drop_query2,drop_query3]

[session.execute(f) for f in drop_tables]

### Close the session and cluster connection¶

In [49]:
session.shutdown()
cluster.shutdown()